# Installation

In [ ]:
!pip install google-cloud-storage

In [ ]:
!pip install gradio

# End to End Flow

In [ ]:
import joblib

Bucket Name

In [ ]:
# @title
bucket_name = "devfolio"

### Importing Model from the bucket.

In [ ]:
#Read Model from the bucket
from io import BytesIO
import joblib
from google.cloud import storage
storage_client = storage.Client()


bucket = storage_client.get_bucket(bucket_name)
blob = bucket.get_blob('prophet_model.joblib')

model_bytes = blob.download_as_bytes()

In [ ]:
# @title
#model = joblib.load('prophet_model.joblib')

In [ ]:
model_bytes_io = BytesIO(model_bytes)
model = joblib.load(model_bytes_io)

### Using the prophet model to predict for the succeeding month (30 days)

In [ ]:
forecast = model.make_future_dataframe(periods=31)
pred = model.predict(forecast)

### Generating the  campagin content using Gemini.

- The  generative model used is gemini-1.5-flash-001.

- Creates an instance of the GenerativeModel class with the specified model name and system instruction. The instruction sets the context for the model, making it act as a hotel marketing specialist.

In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

# Function to generate content based on input text
def generate(text):
    # Initialize Vertex AI with the specified project and location
    vertexai.init(project="c***", location="us-central1")
    model = GenerativeModel(
        "gemini-1.5-flash-001",
        system_instruction=["""You are a personalized Hotel Marketing Specialist that boost hotel occupancy."""]
    )
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 0.1,
        "top_p": 0.95,
    }

    # Define safety settings to filter out harmful content
    safety_settings = [
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
        ),
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
        ),
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
        ),
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
            threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
        ),
    ]

     # Generate content based on the input text using the specified configuration and safety settings
    responses = model.generate_content(
        [text],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True, # Indicates that the content is generated in a streaming manner
    )
    # Process the generated responses
    str_res = []
    for response in responses:
        str_res.append(response.text) # Append each response text to the list
    str_res = ' '.join(str_res) # Join all responses into a single string
    return str_res

### Creating an Gradio interface.

- we set up a web interface where users can select a customer type and generate either an email campaign or a social media post to promote the hotel during periods of historically low bookings.

In [ ]:
import gradio as gr
import joblib
from io import BytesIO

# Average number of rooms booked
#average_rooms_bkd = 37

# Load the model
#model = joblib.load('prophet_model.joblib')

# Hotel features
hotel_feat = {
    "Hotel name": "Stark Tower Hotel and Resorts",
    "Location": "Goa, India",
    "Key Features": ["Ocean-view rooms", "High-speed Wi-Fi in all rooms",
                     "Business lounge with meeting facilities", "Spa and wellness center",
                     "Outdoor pool and kid's play area",
                     "Proximity to major attractions (e.g., local beaches, historic sites)",
                     "On-site restaurant offering local and international cuisines",
                     "Complimentary airport shuttle"]
}

# Function to create email campaign
def create_email_campaign(customer_type):
    # Generate a forecast for the next 31 days
    forecast = model.make_future_dataframe(periods=31)
    pred = model.predict(forecast)
    #print(pred)

    # Calculate the average predicted rooms booked over the forecast period
    pred_avrg_rooms = sum(pred['yhat'][-31:]) / len(pred['yhat'][-31:])
    #print(pred_avrg_rooms)

    # Calculate the percentage difference from the historical average
    avrg_percentage = (pred_avrg_rooms - average_rooms_bkd) / average_rooms_bkd
    #print(avrg_percentage)

    # Identify dates with lower-than-average bookings
    less_booking = pred['ds'][-31:][pred['yhat'][-31:] < 36.5].tolist()
    date_strings = [ts.strftime('%Y-%m-%d') for ts in less_booking]
    #print(date_strings)

    # Craft the email content based on the customer type
    if customer_type == 'Family':
        text = f"""As an expert email marketer, craft a compelling email targeting families to promote our hotel during the dates of {date_strings}. These dates historically have lower-than-average occupancy. Highlight the following hotel features: {hotel_feat}. Develop attractive offers to boost bookings, incorporating any relevant local holidays or events to increase occupancy during this traditionally slower period."""
    elif customer_type == 'Couples':
        text = f"""As an expert email marketer, craft a compelling email targeting couples to promote our hotel during the dates of {date_strings}. These dates historically have lower-than-average occupancy. Highlight the following hotel features: {hotel_feat}. Develop attractive offers to boost bookings, incorporating any relevant local holidays or events to increase occupancy during this traditionally slower period."""
    elif customer_type == 'Business Meet':
        text = f"""As an expert email marketer, craft a compelling email promoting a business meeting at our hotel. Highlight these features: {hotel_feat}. To increase occupancy during historically low booking periods ({date_strings}), develop attractive offers. Consider incorporating any relevant local holidays or events to further incentivize bookings. The goal is to achieve higher occupancy rates during these slower times."""

    response = generate(text)
    return response

# Function to create social media post
def create_social_media_post(customer_type):
    # Generate a forecast for the next 31 days
    forecast = model.make_future_dataframe(periods=31)
    pred = model.predict(forecast)
    #print(pred)


    pred_avrg_rooms = sum(pred['yhat'][-31:]) / len(pred['yhat'][-31:])
    #print(pred_avrg_rooms)
    avrg_percentage = (pred_avrg_rooms - average_rooms_bkd) / average_rooms_bkd
    #print(avrg_percentage)

    # Identify dates with lower-than-average bookings
    less_booking = pred['ds'][-31:][pred['yhat'][-31:] < 36.5].tolist()
    date_strings = [ts.strftime('%Y-%m-%d') for ts in less_booking]

    if customer_type == 'Family':
        text = f"""You are expert in creating an engaging social media post. craft an engaging social media post targeting families, highlighting the following hotel features: {hotel_feat}. The dates {date_strings} have historically shown below-average booking rates. Develop attractive offers and highlight these dates to boost occupancy, incorporating any relevant local holidays or events."""
    elif customer_type == 'Couples':
        text = f"""You are expert in creating an engaging social media post. craft an engaging social media post targeting Couples, highlighting the following hotel features: {hotel_feat}. The dates {date_strings} have historically shown below-average booking rates. Develop attractive offers and highlight these dates to boost occupancy, incorporating any relevant local holidays or events."""
    elif customer_type == 'Business Meet':
        text = f"""You are expert in creating an engaging social media post. craft an engaging social media post targeting Business Meetings, highlighting the following hotel features: {hotel_feat}. The dates {date_strings} have historically shown below-average booking rates. Develop attractive offers and highlight these dates to boost occupancy, incorporating any relevant local holidays or events."""

    response = generate(text)
    return response

# Define customer types
customer_types = ["Family", "Couples", "Business Meet"]

# Create Gradio interface
with gr.Blocks() as demo:
    # Dropdown menu for selecting the customer type
    customer_type = gr.Dropdown(choices=customer_types, label="Customer Type")

    with gr.Row():
        # Button to generate the email campaign
        email_button = gr.Button("Generate Email Campaign")
        social_media_button = gr.Button("Generate Social Media Post")

    #Textbox to display the generated email campaign and social media post
    email_output = gr.Textbox(label="Email Campaign")
    social_media_output = gr.Textbox(label="Social Media Post")

    # Link the email button to the create_email_campaign function
    email_button.click(fn=create_email_campaign, inputs=customer_type, outputs=email_output)

    # Link the social media button to the create_social_media_post function
    social_media_button.click(fn=create_social_media_post, inputs=customer_type, outputs=social_media_output)

# Launch the interface
demo.launch(debug=True)
